In [1]:
% load_ext autoreload
% autoreload 2

import sys
import time
from functools import partial

# to get the ray dashboard, use `pip install 'ray[default]'`
import ray

sys.path.append('../')
from vflow import ModuleSet, Module, AsyncModule, init_args

In [2]:
# for now, the user has to init ray, but we could choose to do it for them in the package
ray.init(num_cpus=4)

2021-07-07 11:19:33,413	INFO services.py:1272 -- View the Ray dashboard at http://127.0.0.1:8265


{'node_ip_address': '10.142.89.30',
 'raylet_ip_address': '10.142.89.30',
 'redis_address': '10.142.89.30:6379',
 'object_store_address': '/tmp/ray/session_2021-07-07_11-19-32_116158_131831/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-07-07_11-19-32_116158_131831/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2021-07-07_11-19-32_116158_131831',
 'metrics_export_port': 62380,
 'node_id': 'd6429e9a49e736165af0f486aaa2f891bd5cb22befefb3102b12d30a'}

In [3]:
def slow_fun(a, b=1):
    time.sleep(1)
    return a + b


# create 4 modules using partials of slow_fun
# the 'module' arg to Module.__init__ must either have a .fit method or be callable
modules = [Module(f'fun{i}', partial(slow_fun, b=i)) for i in range(4)]

slow_set = ModuleSet('slow_set', modules)

# if there's only one arg, it comes wrapped in a list and I can't pass the result of init_args directly to .fit
args = init_args([1], ['a'])[0]

In [4]:
% % time

slow_set(args)

CPU times: user 229 ms, sys: 24.8 ms, total: 254 ms
Wall time: 4 s


{('a', 'slow_set_0'): 1,
 ('a', 'slow_set_1'): 2,
 ('a', 'slow_set_2'): 3,
 ('a', 'slow_set_3'): 4,
 '__prev__': <pcsp.module_set.ModuleSet at 0x7f85105ee190>}

In [5]:
# 1st option to use parallelism is to just use is_async=True when initializing ModuleSet
fast_set = ModuleSet('fast_set', modules, is_async=True)

In [6]:
% % time

fast_set(args)

CPU times: user 96.1 ms, sys: 31.8 ms, total: 128 ms
Wall time: 1.12 s


{('a', 'fast_set_0'): 1,
 ('a', 'fast_set_1'): 2,
 ('a', 'fast_set_2'): 3,
 ('a', 'fast_set_3'): 4,
 '__prev__': <pcsp.module_set.ModuleSet at 0x7f85105d6af0>}

In [7]:
# 2nd option is to use AsyncModules directly
async_modules = [AsyncModule(f'fun{i}', partial(slow_fun, b=i)) for i in range(4)]
fast_set = ModuleSet('fast_set', async_modules)

In [8]:
% % time

fast_set(args)

CPU times: user 44 ms, sys: 20 ms, total: 64 ms
Wall time: 1.01 s


{('a', 'fast_set_0'): 1,
 ('a', 'fast_set_1'): 2,
 ('a', 'fast_set_2'): 3,
 ('a', 'fast_set_3'): 4,
 '__prev__': <pcsp.module_set.ModuleSet at 0x7f85102a3d90>}